In [1]:
import numpy as np
import pandas as pd
from skbio import DistanceMatrix
from skbio.stats.ordination import pcoa
from biom import load_table
import random
import pickle
from sklearn.mixture import BayesianGaussianMixture
import matplotlib.pyplot as plt
from scipy.spatial.distance import braycurtis

# Set seed for reproducibility
random.seed(531)
np.random.seed(531)

# Load data (replace with actual paths to the .biom or data files)
fgfp_10Kg = load_table("fgfp_10Kg.biom")
lcpm_10Kg = load_table("lcpm_10Kg.biom")

# Check for differences in taxa names
taxa_fgfp = set(fgfp_10Kg.ids(axis='observation'))
taxa_lcpm = set(lcpm_10Kg.ids(axis='observation'))
diff_lcpm_fgfp = taxa_lcpm.difference(taxa_fgfp)
diff_fgfp_lcpm = taxa_fgfp.difference(taxa_lcpm)

# Add cohort information and merge datasets
fgfp_10Kg.add_metadata({'Cohort': 'FGFP'}, axis='sample')
lcpm_10Kg.add_metadata({'Cohort': 'LCPM'}, axis='sample')
lcpm_fgfp_10Kg = fgfp_10Kg.merge(lcpm_10Kg)

# Perform PCoA analysis using Bray-Curtis distance
def calculate_bray_curtis_distance(matrix):
    return DistanceMatrix(braycurtis(matrix))

matrix = lcpm_fgfp_10Kg.matrix_data.toarray().T
bray_dm = calculate_bray_curtis_distance(matrix)
pcoa_results = pcoa(bray_dm)
# Plot PCoA results
plt.scatter(pcoa_results.samples['PC1'], pcoa_results.samples['PC2'], c=lcpm_fgfp_10Kg.metadata_to_dataframe()['Cohort'].factorize()[0])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCoA of Samples Colored by Cohort")
plt.show()

# Process OTU Table
genus_table = pd.DataFrame(lcpm_fgfp_10Kg.matrix_data.toarray(), index=lcpm_fgfp_10Kg.ids('observation'), columns=lcpm_fgfp_10Kg.ids('sample')).T
print("Dimensions of Genus Table:", genus_table.shape)
print("Column Sums Range:", genus_table.sum(axis=0).min(), "-", genus_table.sum(axis=0).max())
print("Row Sums Range:", genus_table.sum(axis=1).min(), "-", genus_table.sum(axis=1).max())

# Add pseudocount to OTU table and DMM enterotyping
otu_table = genus_table + 1e-9
enterotype_df = pd.DataFrame(index=otu_table.columns)

# Perform Dirichlet Multinomial Mixture (DMM) clustering
n_components = 6
dmn_models = [BayesianGaussianMixture(n_components=k).fit(otu_table.T) for k in range(1, n_components + 1)]

# Save models
with open("fgfp_lcpm_10K_et_dmms.pkl", "wb") as f:
    pickle.dump(dmn_models, f)

# Select optimal DMM model based on AIC, BIC, etc.
aics = [model.aic(otu_table.T) for model in dmn_models]
bics = [model.bic(otu_table.T) for model in dmn_models]
min_aic_model = dmn_models[np.argmin(aics)]
min_bic_model = dmn_models[np.argmin(bics)]

# Plot AIC and BIC for the number of components
plt.plot(range(1, n_components + 1), aics, label='AIC', marker='o')
plt.plot(range(1, n_components + 1), bics, label='BIC', marker='o')
plt.xlabel('Number of Dirichlet Components')
plt.ylabel('Model Fit')
plt.legend()
plt.show()

# Assign clusters and create enterotype dataframe
enterotypes = min_aic_model.predict(otu_table.T)
enterotype_df['ET_clusters'] = enterotypes

# Cluster naming for specific genera
taxa_list = ["g_Bacteroides", "g_Faecalibacterium", "g_Prevotella", "g_Alistipes", "g_Ruminococcus", "g_Methanobrevibacter"]
genus_summary = genus_table[taxa_list].groupby(enterotype_df['ET_clusters']).median()
genus_summary.sort_values(by="g_Prevotella", inplace=True)
print("Enterotype Summary:", genus_summary)

# Map clusters to enterotypes (Prev, Bact1, Bact2, Rum)
et_output = pd.DataFrame(enterotypes, index=genus_table.index, columns=["ET_clusters"])
et_names = et_output["ET_clusters"].replace({genus_summary.idxmax()["g_Prevotella"]: "Prev", genus_summary.idxmax()["g_Bacteroides"]: "Bact2", genus_summary.idxmin()["g_Bacteroides"]: "Bact1", genus_summary.idxmax()["g_Ruminococcus"]: "Rum"})
et_output["ET_names"] = et_names
print("Final Enterotyping Table:\n", et_output.head())


ModuleNotFoundError: No module named 'skbio'